In [ ]:
# Create shapes dataset
# Remove corners from each shape and save to new folder
# Remove edges from each shape and save to new folder

In [18]:
import cv2
import numpy as np
import os
import math

classes = [1, 3, 4, 5, 6, 7, 8]
num_per_class = 2
imgsize = 200
min_radius = 20
thickness = 2
bg_color = (255, 255, 255)
fg_color = (0, 0, 0)
saveto = "./images/shapes/"
corner_hide = 0 # Fraction of shape radius, such that a circle of bg_color will be drawn around each corner with this radius to hide the corner
edge_hide = 0.3 # Fraction of shape radius, such that a circle of bg_color will be drawn around center of each edge with this radius to hide the edge

for side in classes:
    for k in range(num_per_class):
        suffix = 'whole' # Save image with suffix to indicate corner_hide, edge_hide, or whole
        # Create a blank image
        img = np.zeros((imgsize, imgsize, 3), np.uint8)
        img[:] = bg_color
        # Randomly select a point on the image
        x = np.random.randint(min_radius + math.ceil(thickness/2) + 1, imgsize-min_radius - math.ceil(thickness/2))
        y = np.random.randint(min_radius + math.ceil(thickness/2) + 1, imgsize-min_radius - math.ceil(thickness/2))
        # Get max radius of polygon
        max_radius = min(x, y, imgsize - x, imgsize - y)
        # Randomly select a radius
        radius = np.random.randint(min_radius, max_radius)
        # Randomly choose a starting angle
        angle = np.random.randint(0, 360)
        angle = angle * np.pi / 180 # Convert to radians
        # Get list of points in polar coordinates
        if side > 2:
            angles = [angle + 2 * np.pi * i / side for i in range(side)]
            points = np.array([(x + radius * np.cos(angle), y + radius * np.sin(angle)) for angle in angles], np.int32)
            # Draw polygon
            img = cv2.polylines(img, [points], True, fg_color, thickness)
            # Hide corners, if requested
            if corner_hide > 0:
                suffix = 'corner_hide'
                for i in range(side):
                    cv2.circle(img, (points[i][0], points[i][1]), math.ceil(corner_hide * radius), bg_color, -1)
            # Hide edges, if requested
            if edge_hide > 0:
                suffix = 'edge_hide'
                for i in range(side):
                    cv2.circle(img, (int((points[i][0] + points[(i + 1) % side][0]) / 2), int((points[i][1] + points[(i + 1) % side][1]) / 2)), math.ceil(edge_hide * radius), bg_color, -1)
        elif side == 1:
            # Draw circle
            img = cv2.circle(img, (x, y), radius, fg_color, thickness)
        else:
            print("Invalid side number")
        # Save image
        cv2.imwrite(f'{saveto}{side}_{k}_{suffix}.png', img)